# Taller 2: Aprendizaje Supervisado – Modelos de Clasificación
Integrantes: Zada Riquelme y Vania Reyes.

------------------------------------------------------------------------------------------------------------------------

# Actividad 1

Para el preprocesamiento de datos, llevamos a cabo una limpieza de columnas y rellenamos los datos perdidos con ceros, esta última decisión la tomamos al ser que la columna INS_1SEM e INS_2SEM proporciona información de las asignaturas que se tomaron por cada semestre cada alumno, lo que nos llevó a conjeturar que los promedios que se encontraban vacíos es porque el estudiante no los inscribió por lo que nos pareció más prudente reemplazarlos por ceros que por la media, la cual era otra de nuestras opciones.
A continuación mencionamos las columnas que determinamos que no son relevantes para este objetivo del taller: ACTF_1SEM_R, ACTF_2SEM_R, descripcion_situacion_egreso_postulante, nombre_secretaria_admision, descripcion_jefe_familia, descripcion_nivel_educacion_padre, descripcion_nivel_educacion_madre, descipcion_tipo_organismo_trabajan_padre, descripcion_tipo_organismo_trabajan_madre, descripcion_ocupacion_principal_padre, descripcion_ocupacion_principal_madre, descripcion_rama_actividad_padre, descripcion_rama_actividad_madre, cuantos_trabajan_grupos_familiar, cuantos_estudian_grupo_familiar, cuantos_estudian_grupo_pre_basica, cuantos_estudian_grupo_media_1_3, cuantos_estudian_grupo_media_4, cuantos_estudian_grupo_otras.

# Actividad 2

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

archivo = 'Taller_2_Titulacion_DatosTaller.csv'
datos = pd.read_csv(archivo, encoding='latin-1', delimiter=';')

X = datos.iloc[:, :-1]
y = datos.iloc[:, -1]
X = X.drop('Id', axis=1)

columnas_numericas = X.select_dtypes(include=['int64', 'float64']).columns
columnas_categoricas = X.select_dtypes(include=['object']).columns

pipeline_numerico = Pipeline([
    ('imputador', SimpleImputer(strategy='median')),
    ('escalador', StandardScaler())
])

pipeline_categorico = Pipeline([
    ('imputador', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocesador = ColumnTransformer([
    ('num', pipeline_numerico, columnas_numericas),
    ('cat', pipeline_categorico, columnas_categoricas)
])

X_procesado = preprocesador.fit_transform(X)
y = pd.get_dummies(y, drop_first=True).values
X_procesado = X_procesado.astype('float32')
y = y.astype('float32')

def crear_modelo():
    modelo = Sequential()
    modelo.add(Dense(512, activation='relu', input_shape=(X_procesado.shape[1],)))
    modelo.add(Dropout(0.5))
    modelo.add(Dense(512, activation='relu'))
    modelo.add(Dropout(0.5))
    modelo.add(Dense(256, activation='relu'))
    modelo.add(Dense(1, activation='sigmoid'))
    modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return modelo

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

resultados = []

for fold_index, (train_index, val_index) in enumerate(kf.split(X_procesado, y)):
    print(f'Entrenando fold {fold_index + 1}/{num_folds}...')
    
    X_train, X_val = X_procesado[train_index], X_procesado[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    modelo = crear_modelo()
    
    parada_temprana = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)
    
    historia = modelo.fit(X_train, y_train, epochs=200, batch_size=64,
                          validation_data=(X_val, y_val), callbacks=[parada_temprana], verbose=1)
    
    resultados.append(historia.history['val_accuracy'][-1])

resultados = np.array(resultados)
print(f'Precisión promedio en validación cruzada K-fold: {np.mean(resultados):.4f} ± {np.std(resultados):.4f}')


Entrenando fold 1/5...
Epoch 1/200


c:\Users\Vania\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.6534 - loss: 0.6033 - val_accuracy: 0.7070 - val_loss: 0.5454
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7236 - loss: 0.5324 - val_accuracy: 0.7252 - val_loss: 0.5238
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7554 - loss: 0.5028 - val_accuracy: 0.7169 - val_loss: 0.5388
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7619 - loss: 0.4931 - val_accuracy: 0.7550 - val_loss: 0.5158
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7900 - loss: 0.4600 - val_accuracy: 0.7401 - val_loss: 0.5183
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8102 - loss: 0.4264 - val_accuracy: 0.7202 - val_loss: 0.5347
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.8011 - loss: 0.4193 - val_accuracy: 0.7268 - val_loss: 0.5401
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.8055 - loss: 0.4052 - val_accuracy: 0.7020 - val_l

# Actividad 3

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as np  # Importar numpy para usar .ravel()

# Define models and parameter grids
models = {
    'RandomForest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20]
        }
    },
    'SVM': {
        'model': SVC(random_state=42),
        'params': {
            'kernel': ['linear', 'rbf'],
            'C': [1, 10, 100]
        }
    }
}

# Function to perform grid search and return best model
def find_best_model(model, params, X_train, y_train, X_test, y_test):
    # Convertir y_train y y_test a arrays 1D usando .ravel()
    y_train = np.ravel(y_train)
    y_test = np.ravel(y_test)
    
    grid_search = GridSearchCV(estimator=model, param_grid=params, cv=3, scoring='accuracy', verbose=1)
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    
    # Evaluate on test set
    y_pred = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    return best_model, best_params, accuracy

# Perform grid search for each model
results = {}
for model_name, config in models.items():
    best_model, best_params, accuracy = find_best_model(config['model'], config['params'], X_train, y_train, X_test, y_test)
    
    results[model_name] = {
        'best_model': best_model,
        'best_params': best_params,
        'accuracy': accuracy
    }
    

# Select the model with the highest accuracy
best_model_name = max(results, key=lambda x: results[x]['accuracy'])
best_model = results[best_model_name]['best_model']
best_params = results[best_model_name]['best_params']
best_accuracy = results[best_model_name]['accuracy']

print(f"Best model overall: {best_model_name}")
print(f"Best parameters: {best_params}")
print(f"Accuracy on test set: {best_accuracy:.4f}")


Fitting 3 folds for each of 9 candidates, totalling 27 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best model overall: RandomForest
Best parameters: {'max_depth': None, 'n_estimators': 200}
Accuracy on test set: 0.7318


In [7]:
# Assuming X_train, X_test, y_train, y_test are already prepared and model is trained

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Define and train the best model
best_model = RandomForestClassifier(max_depth=None, n_estimators=200, random_state=42)
best_model.fit(X_train, y_train)

# Evaluate on test set
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Best model selected: RandomForest')
print(f'Best parameters: {{"max_depth": None, "n_estimators": 200}}')
print(f'Accuracy on test set: {accuracy:.4f}')


c:\Users\Vania\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best model selected: RandomForest
Best parameters: {"max_depth": None, "n_estimators": 200}
Accuracy on test set: 0.7318


# Actividad 4

In [9]:
import pandas as pd

# Cargar el conjunto de datos de evaluación
archivo_evaluacion = 'Taller_2_Titulacion_Evaluación.csv'
datos_evaluacion = pd.read_csv(archivo_evaluacion, encoding='latin-1', delimiter=';')

# Guardar los Ids para el archivo de salida
ids_evaluacion = datos_evaluacion['Id']

# Preprocesar datos de evaluación similar al conjunto de entrenamiento
X_evaluacion = datos_evaluacion.drop('Id', axis=1)  # Eliminar columna 'Id'

# Aplicar el preprocesamiento ya definido anteriormente
X_evaluacion_processed = preprocessor.transform(X_evaluacion)

# Realizar predicciones utilizando el mejor modelo seleccionado (RandomForest)
predicciones_evaluacion = best_model.predict(X_evaluacion_processed)

# Convertir predicciones a formato 'SÍ' o 'NO'
predicciones_evaluacion_etiquetas = ['SÍ' if pred == 1 else 'NO' for pred in predicciones_evaluacion]

# Crear DataFrame para el archivo de salida
resultado_df = pd.DataFrame({'Id': ids_evaluacion, 'Predicción': predicciones_evaluacion_etiquetas})

# Guardar resultados en archivo de texto
archivo_salida = 'clasificación_título.txt'
resultado_df.to_csv(archivo_salida, sep=';', index=False)

print(f'Archivo "{archivo_salida}" generado exitosamente.')


Archivo "clasificación_título.txt" generado exitosamente.
